In [87]:
import numpy as np
import pandas as pd
import itertools as it
import random as rd

#System
import os 
from glob import glob

import re

#Music Analysis
import music21 as m21

#Encodage
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#LSTM
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

m21.environment.set('musescoreDirectPNGPath', '/Applications/MuseScore 3.app/contents')

## Encoding DB

In [2]:
def sentenceEncoding(sentence, order = "character", sepTime = False) :
    #retourne encodage one hot de la phrase, integer encode de la phrase et la taille du vocabulaire
    #Ordre = chara ou word
    if order == "character" :
        if sepTime :
            sentence = sentence.replace("°", "_")
        token = sentence.split("_")
    elif order == "word" :
        token = sentence.split(" ")
        #A finir
    
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(token)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    vocabLength = int(max(integer_encoded))
    return onehot_encoded, integer_encoded, token, vocabLength

## Decoding and midi creation

In [3]:
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

def word2chord(word, duration = True) :
    if "Start" in word :
        word = word[6:-1]
    elif "End" in word : 
        if len(word)==3 :
            return False
        word = word[1:-3]
    else :
        word = word[1:-1]
        
    chordTemp = []
    for letter in word.split("_") :
        noteTemp, durationTemp = letter.split("°")
        duration = m21.duration.Duration(convert_to_float(durationTemp))
        note = m21.note.Note(noteTemp, duration = duration)
        chordTemp.append(note)
    chord = m21.chord.Chord(chordTemp)
    return chord

def token2midi(token):
    s = m21.stream.Stream()
    flag = False
    i=0
    while i <len(token) :
        chordTemp = []
        while token[i] != " " and not("Start" in token[i]) and not("End" in token[i]) and i <len(token)-1 :
            word = token[i]
            flag = True
            noteTemp, durationTemp = word.split("°")
            duration = m21.duration.Duration(convert_to_float(durationTemp))
            note = m21.note.Note(noteTemp, duration = duration)
            chordTemp.append(note)
            i+=1
        if flag :
            chord = m21.chord.Chord(chordTemp)
            s.append(chord)
            flag = False
        i+=1
            
    return s

def sentence2midi(sentence) :
    s = m21.stream.Stream()
    for wordTemp in sentence.split(" ")[5:-5] :
        thisChord = word2chord(wordTemp)
        if thisChord :#On vérifie que l'accord n'est pas vide
            s.append(thisChord)
    return s

def token2sentence(token, order="character", sepTime = False) :
    sentence=""
    if order == "character" :
        if sepTime :
            
                if "Start" in word :
                    sentence+=word+"_"
                elif "." in word : # C'est une durée
                    sentence+="°"+word+"_"
                elif " " in word:# C'est un espace
                    sentence+=word+"_"
                else :
                    sentence+=word
        else :
            for word in token :
                sentence+=word+"_"
        return sentence[:-1]
    elif order == "word" :
        for word in token :
            sentence+=word+" "
    return sentence[:-1]
    

In [186]:
musicDB = pd.read_pickle("keithKoln.pkl")
musicDB

,Artist,File,Piece
0,Test2,keithkoln.mid,_B-3°0.25_ _F4°0.5_ _E-4°5.25_ _B-3°8.25_ _C4°...


In [201]:
classicSentence = ""
for sentence in musicDB["Piece"] :
    classicSentence+=sentence+" "
    
classicSentence = classicSentence[:10000]
classicSentence

'_B-3°0.25_ _F4°0.5_ _E-4°5.25_ _B-3°8.25_ _C4°8.5_ _F3°2.0_F2°0.5_ _C3°7.5_ _F3°2.0_ _F3°2.0_ _F3°2.0_ _B-3°0.25_F3°2.0_ _F4°0.5_ _E-4°0.25_ _B-3°0.25_ _F4°0.25_C4°3.0_ _G4°0.25_ _G4°0.0_A-4°0.5_F3°2.0_ _G4°0.25_ _F4°0.5_ _E-4°5.5_ _G2°2.25_ _G3°0.25_C2°1.0_D3°1.0_C2°1.0_D3°1.0_ _C4°0.5_ _B-3°0.25_ _F3°0.25_ _G3°4.0_ _F3°2.0_C3°1.5_ _G2°1.25_ _C2°3.5_D3°2.0_F3°0.0_ _C4°0.25_G2°2.5_ _C4°0.25_ _E-4°0.5_D3°0.0_ _E-4°0.25_ _E-4°0.25_ _C4°0.25_ _C4°0.25_ _C4°0.5_ _G3°0.0_E-2°0.25_ _A-3°0.0_B-3°0.5_C3°1.0_B-1°1.0_E-3°1.0_C3°1.0_B-1°1.0_E-3°1.0_C3°1.0_B-1°1.0_E-3°1.0_ _B-3°0.0_ _G3°0.5_ _G3°1.0_B-3°1.0_G3°1.0_B-3°1.0_ _E-3°0.5_B-3°0.0_ _E-3°0.0_B-3°2.0_C3°1.0_E-3°1.0_C3°1.0_E-3°1.0_ _E-3°0.75_ _E-2°0.25_ _E-3°2.5_A-1°1.5_C3°2.0_ _E-2°0.5_ _C3°0.5_ _E-3°0.25_ _E-3°0.25_ _F3°0.25_ _G3°0.25_F2°0.25_ _B-3°0.5_F3°3.0_B-1°1.0_D3°1.0_B-1°1.0_D3°1.0_ _D4°0.0_ _E-4°0.25_ _D4°0.5_ _B-3°0.25_ _C4°0.25_ _B-3°3.0_D3°2.0_ _F3°0.5_ _F3°0.5_ _F3°0.5_B-1°1.0_D3°2.0_ _F3°0.5_ _F3°0.5_ _B-1°0.25_ _B-3°1.0_F3°1

In [188]:
net_input, integer_encoded, token, vocabLength = sentenceEncoding(classicSentence, "character")

In [189]:
token

1630

## Modele LSTM

In [190]:
import sys
import re 
import numpy as np 
import pandas as pd
import music21
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils

In [191]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    # Extract the unique pitches in the list of notes.
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return network_input, network_output

In [195]:
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    #model.add(Dropout(0.2))
    #model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    #model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'Models/weights_Bach_Fugue_20ep.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])
    

def train_network(ep = 10):
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = ep
    
    notes = token
    print('Notes processed')
    
    n_vocab = vocabLength+1 ## /!\ Je comprends pas pourquoi il ya besoins d'ajouter 1
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    #return model
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')

In [196]:
train_network(40)

Notes processed
Vocab generated
Input and Output processed
Model created
Training in progress
Epoch 1/40
48/48 [==============================] - 5s 106ms/step - loss: 3.9676
Epoch 2/40
48/48 [==============================] - 6s 119ms/step - loss: 3.6624
Epoch 3/40
48/48 [==============================] - 7s 155ms/step - loss: 3.5742
Epoch 4/40
48/48 [==============================] - 9s 184ms/step - loss: 3.4112
Epoch 5/40
48/48 [==============================] - 9s 185ms/step - loss: 3.2310
Epoch 6/40
48/48 [==============================] - 6s 129ms/step - loss: 3.0202
Epoch 7/40
48/48 [==============================] - 6s 131ms/step - loss: 2.8161
Epoch 8/40
48/48 [==============================] - 8s 168ms/step - loss: 2.5720
Epoch 9/40
48/48 [==============================] - 6s 133ms/step - loss: 2.2998
Epoch 10/40
48/48 [==============================] - 7s 148ms/step - loss: 2.0455
Epoch 11/40
48/48 [==============================] - 9s 195ms/step - loss: 1.8196
Epoch 12/40
4

In [197]:
def get_inputSequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in tqdm(range(500)):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction_input = np.asarray(prediction_input).astype(np.float32)

        prediction = model.predict(prediction_input, verbose=0)
        
        # Predicted output is the argmax(P(h|D))
        
        index = np.argmax(prediction)
        
        #/!\ Tester de ne pas prendre argmax mais un random choice  
        
        # Mapping the predicted interger back to the corresponding note
        result = int_to_note[index]
        # Storing the predicted output
        prediction_output.append(result)

        pattern.append(index)
        # Next input to the model
        pattern = pattern[1:len(pattern)]

    print('Notes Generated...')
    return prediction_output

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    notes = token

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = vocabLength+1
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('Models/weights_Bach_Fugue_20ep.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    return prediction_output

In [198]:
a = generate()

Initiating music generation process.......
Loading Model weights.....
Model Loaded
Generating notes........
Notes Generated...


In [199]:
s=token2midi(a)

In [200]:
s.show()